In [1]:
from src.parser import get_raw_segments
from src.classes import ScoreAnnot 
from src.annot_types import string_to_instrument
import pandas as pd
import numpy as np


In [3]:
meta_file = './dataset/new_metadata.csv'
df_meta = pd.read_csv(meta_file)
'''
path = './dataset/annotations/'
for piece in range(df_meta.shape[0]):
    piece_name = df_meta['score'][piece].split('/')[-1].split('.')[0]
    df_meta['annot_npy'][piece] = path+piece_name+'.npy'

df_meta.to_csv(meta_file)'''
print(df_meta)

    Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0   composer  \
0              0             0             0           0     mozart   
1              1             1             1           1     mozart   
2              2             2             2           2     mozart   
3              3             3             3           3     mozart   
4              4             4             4           4      haydn   
5              5             5             5           5      haydn   
6              6             6             6           6      haydn   
7              7             7             7           7      haydn   
8              8             8             8           8      haydn   
9              9             9             9           9  beethoven   
10            10            10            10          10  beethoven   
11            11            11            11          11  beethoven   
12            12            12            12          12  beethoven   
13    

In [13]:
def make_label_array(score: ScoreAnnot)->np.array:
    n_inst = len(score.get_inst_list())
    inst_list = score.get_inst_list()
    segments = score.get_list_segments()

    end = 0
    arr = []
    #iterate through each segment
    for seg in segments:
        layer_list = seg.get_list_layers()
        role_dict = dict( (string_to_instrument(inst), None) for inst in inst_list ) # In hob103, there will be 2 VIOLIN1, but since their roles are identical, it will be fine
        #print( 'Before editting: role_dict = ', role_dict )

        # fill in the role of each instrumnet
        for layer in layer_list:
            role_one_hot = np.array( layer.get_role().get_role_three_bool() )
            for inst in layer.get_ensemble().get_list_components():
                role_dict[inst] = role_one_hot
        #print( 'After editting: role_dict = ', role_dict )

        # make the label numpy array
        a_measure = [] #shape = (n_inst, 3)
        for inst in inst_list:
            inst = string_to_instrument(inst)
            if role_dict[inst] is not None:
                a_measure.append( role_dict[inst] )
            else: # instrument not in any layers -> not making sound
                a_measure.append( np.zeros(3, dtype=bool) ) # MAYBE WE CAN HANDLE IT DIFFERENTLY, NOW IT IS IDENTICAL TO role='None'
        a_measure_arr = np.array(a_measure)
        #print(a_measure_arr.shape)
        #print(a_measure_arr)

        #if there is a gap (empty bars)
        if seg.get_measure_beg() != end+1:
            #print(f'There is a gap at {end}')
            for i in range(end+1, seg.get_measure_beg()):
                arr.append( np.zeros((n_inst, 3),dtype=bool ) ) #append empty labels 
        for i in range( seg.get_measure_beg(), seg.get_measure_end()+1 ):
            arr.append( a_measure_arr )
        end = seg.get_measure_end()
        
    # after iterating through all segments
    piece_label = np.array( arr ) # shape = ( n_measures, n_inst, 3 )
    print(piece_label.shape)
    return piece_label

In [76]:
file_path = df_meta['annotation'][0]
l = get_raw_segments(file_path)
score = ScoreAnnot(l)
inst_name_list = [ string_to_instrument(inst) for inst in score.get_inst_list() ]
print(inst_name_list)
make_label_array(score)

[FLUTE, OBOE, FAGOTT, HORN, TRUMPET, TIMPANI, VIOLIN1, VIOLIN2, VIOLA, CELLO, CONTREBASS]
[[False False  True]
 [False False  True]
 [False False  True]
 [False False  True]
 [False False  True]
 [False False  True]
 [False False  True]
 [False False  True]
 [False False  True]
 [False False  True]
 [False False  True]]
[[ True False False]
 [ True False False]
 [ True False False]
 [False  True  True]
 [False  True  True]
 [False  True  True]
 [ True False False]
 [ True False False]
 [ True False False]
 [ True False False]
 [ True False False]]
[[False False False]
 [False False False]
 [False  True False]
 [False  True  True]
 [False  True  True]
 [False  True  True]
 [False  True False]
 [False  True False]
 [False  True False]
 [False  True False]
 [False  True False]]
[[False False  True]
 [False False  True]
 [False False  True]
 [False False  True]
 [False False  True]
 [False False False]
 [False False  True]
 [False False  True]
 [False False  True]
 [False False  True]
 [Fa

array([[[False, False,  True],
        [False, False,  True],
        [False, False,  True],
        ...,
        [False, False,  True],
        [False, False,  True],
        [False, False,  True]],

       [[ True, False, False],
        [ True, False, False],
        [ True, False, False],
        ...,
        [ True, False, False],
        [ True, False, False],
        [ True, False, False]],

       [[False, False, False],
        [False, False, False],
        [False,  True, False],
        ...,
        [False,  True, False],
        [False,  True, False],
        [False,  True, False]],

       ...,

       [[ True, False, False],
        [ True, False, False],
        [ True, False, False],
        ...,
        [ True, False, False],
        [ True, False, False],
        [ True, False, False]],

       [[ True, False, False],
        [ True, False, False],
        [ True, False, False],
        ...,
        [ True, False, False],
        [ True, False, False],
        [ True,

In [16]:
for piece in range(df_meta.shape[0]):

    file_path = df_meta['annotation'][piece]
    l = get_raw_segments(file_path)
    score = ScoreAnnot(l)
    #inst_name_list = [ string_to_instrument(inst) for inst in score.get_inst_list() ]
    #print(inst_name_list)
    piece_label = make_label_array(score)
    #print( piece_label.shape[0] == int( df_meta['n_measures'][piece] ) )
    #print( piece_label.shape[0], ' ',int( df_meta['n_measures'][piece] ) )
    np.save( df_meta['annot_npy'][piece], piece_label )

(302, 11, 3)
(309, 11, 3)
(299, 11, 3)
(313, 11, 3)
(202, 18, 3)
(289, 16, 3)
(351, 18, 3)
(229, 18, 3)
(294, 18, 3)
(297, 12, 3)
(360, 12, 3)
(691, 15, 3)
(498, 12, 3)
(502, 12, 3)
(512, 10, 3)
(450, 12, 3)
(373, 12, 3)
(547, 13, 3)


In [5]:
score = ScoreAnnot(l)
print(score.get_inst_list())
print(len(score.list_segments))
print( score.list_segments[2].list_layers[1] )

['Fl', 'Ob', 'Fg', 'Hrn', 'Trp', 'Timp', 'Vln1', 'Vln2', 'Vla', 'Vc', 'Cb']
220
{
  "BlendLayer": {
    "internal_bar_beg": 3,
    "internal_bar_end": 3,
    "same_function": true,
    "identifier": "s",
    "function": {
      "role": "SPARSE",
      "relation": "UNISSON"
    },
    "ensemble": {
      "list_components": [
        "HORN",
        "TRUMPET",
        "TIMPANI"
      ]
    }
  }
}
